### Intro to programming model -  DSPy Laboratory

In [3]:
import dspy

from dspy import OpenAI
from dspy import (
    Predict,
    settings,
    ChainOfThought,
    ChainOfThoughtWithHint,
    MultiChainComparison,
    Prediction,
    ProgramOfThought,
    ReAct
)
from dspy import GROQ
from rich import print
from dotenv import dotenv_values

secret = dotenv_values('../../.secret')
# gpt_turbo = OpenAI('gpt-3.5-turbo', api_key=secret['OPEN_AI_API_KEY'], model_type="chat")
# settings.configure(lm=gpt_turbo)


groq = dspy.GROQ(model='llama3-70b-8192', api_key=secret['GROQ_API_KEY'], max_tokens=2000)
dspy.settings.configure(lm=groq, )





In [4]:
from dspy.datasets import DataLoader


dl = DataLoader()

data = dl.from_huggingface(
    "gsm8k", "main",
    fields=("question", "answer"),
    input_keys=("question",),
    split="train[:3]"
)

data

[Example({'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}) (input_keys={'question'}),
 Example({'question': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?', 'answer': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10'}) (input_keys={'question'}),
 Example({'question': 'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?', 'answer': "In the beginning, Betty has only 100 / 2 = $<<100/2=5

In [5]:
import dsp
import dspy
from dspy.datasets import DataLoader
from dspy.experimental import Synthesizer, SynthesizerArguments

config = SynthesizerArguments()
synth = Synthesizer(config=config)
synth_data = synth.generate(ground_source=data, num_data=1)
synth_data



Generating Synthetic Data: 100%|██████████| 1/1 [00:01<00:00,  1.57s/it]


[Example({'question': 'Tom has 15 boxes of crayons in his art class. Each box contains 8 packs of crayons. How many packs of crayons does Tom have in total?\n\nAnswer: 120 packs of crayons', 'answer': 'question: Tom has 15 boxes of crayons in his art class. Each box contains 8 packs of crayons. How many packs of crayons does Tom have in total?\nanswer: 120'}) (input_keys={'question'})]

In [ ]:
pred = Predict("question -> answer", temperature=0.3)
response = pred(question="Why does popcorn pop?")
print(response)

In [ ]:
predict = Predict("question -> answer_as_json")
print(predict)

In [ ]:
response = predict(question="Provide 3 method of statistical inference")
print(response.answer_as_json)

#### Chain Of Thought - Technique (reasoning)

In [ ]:
cot_qa = ChainOfThought("question -> answer", temperature=0.3)
print(cot_qa)

In [ ]:
response = cot_qa(question="What year was Cristobal Colon imprisoned and won the novel award?")

In [ ]:
print(response)

#### Chain Of Thought with Hint - Technique (reasoning with hints)

In [ ]:
question = "In the legal framework, how many weekly working hours are established for the call center industry?"
hint = "Call centers are nearshore industries based in the Caribbean and their clients are in the United States."

coth_qa = ChainOfThoughtWithHint("question -> answer")
response = coth_qa(question=question, hint=hint)
print(response)

#### Multi Chain Comparison - technique of "students" and final reasonable answer

In [ ]:
completions = [
    Prediction(rationale="Yes, when I visit my brother I can identify.", answer="personality identification"),
    Prediction(rationale="a person who is migrating from one country to another", answer="personality migration"),
    Prediction(rationale="bad friend", answer="fake personality"),
]

multichain = MultiChainComparison("question -> answer")

response = multichain(question="distinguishes an individual from others by describing feelings, thinking, and behaviors", completions=completions)

print(response)

#### ReAct (reasoning and acting)


In [ ]:
import dspy
from dspy import ReAct, Signature
from rich import print

# Define a simple signature for basic question answering
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Pass signature to ReAct module
react_module = dspy.ReAct(BasicQA)

# Call the ReAct module on a particular input
question = 'Aside from the Apple Remote, what other devices can control the program Apple Remote was originally designed to interact with?'
question = 'What is the real name of Batman?'
result = react_module(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ReAct process): {result.answer}")

In [ ]:
agent = dspy.ReAct("question -> answer")
agent(question="What's up!")

In [ ]:
agent(question="Could you help me with some basic math?")

### Data & Metrics

In [ ]:
from dspy import Example

ex1 = Example(question="What's position are open now?", answer="Agent Call Center", hint="Consult data base for updates on position jobs.", table_ref="hr_jobs")

print(ex1)

In [ ]:
ex2 = Example(question="According to company rules, what is the most valuable employee?", answer="Sales Agent").with_inputs("company_rules")
print(ex2)

In [ ]:
ex2.inputs(), ex2.labels()

#### Metrics

### Optimizers